In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
from openvino.inference_engine import IECore
import easyocr 
ie  = IECore()
Video_path = r'data_videos\License\stock-footage-chisinau-moldova-february-slow-street-traffic-during-snowy-winnter-at-at-dirty-roads-of.webm'
def show(image):
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

video = cv2.VideoCapture(Video_path)

In [29]:
def detect_license_plates(video_path):
    license_plate_xml = r'models\vehicle-license-plate-detection-barrier-0106\vehicle-license-plate-detection-barrier-0106.xml'
    license_plate_bin = r'models\vehicle-license-plate-detection-barrier-0106\vehicle-license-plate-detection-barrier-0106.bin'
    net = ie.read_network(model=license_plate_xml, weights=license_plate_bin)
    ext = ie.load_network(network=net, device_name='CPU')
    
    video_capture = cv2.VideoCapture(video_path)
    license_plate_images = []
    frame_width = int(video_capture.get(3))
    frame_height = int(video_capture.get(4))
    
    size = (frame_width, frame_height)

    writer = cv2.VideoWriter('test.mp4', 
                            cv2.VideoWriter_fourcc(*'MP4V'),
                            30,size)
    extracted_text = []
    while True:
        ret, frame = video_capture.read()
        
        if not ret:
            break
        
        resized_frame = cv2.resize(frame, (300, 300))
        image_for_inference = np.transpose(resized_frame, (2, 0, 1))
        image_for_inference = np.expand_dims(image_for_inference, axis=0)
        results = ext.infer(inputs={'Placeholder': image_for_inference})
        
        # Retrieve the output blob
        output_blob = next(iter(net.outputs))
        
        # Process the output
        output = results[output_blob]

        boxes = output[0][0]
        
        if output.shape[0] == 0:
            print("No license plates detected.")
        else:
            for box in boxes:
                confidence = box[2]
                lable = box[1]
                
                if confidence > 0.005 and lable==2.0: 
                    # Filter detections based on confidence threshold
                    x_min = int(box[3] * frame.shape[1])
                    y_min = int(box[4] * frame.shape[0])
                    x_max = int(box[5] * frame.shape[1])
                    y_max = int(box[6] * frame.shape[0])
                    cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

                    license_plate_image = frame[y_min:y_max, x_min:x_max]
                    # try:
                    #     image_for_inference = cv2.cvtColor(license_plate_image, cv2.COLOR_BGR2GRAY)
                    #     reader = easyocr.Reader(['en'],gpu=False)
                        
                    #     results = reader.readtext(image_for_inference)
                    #     for detection in results:
                    #             box, text, confidence = detection
                    #             print(text)
                    #             extracted_text.append(text)
                    # except:
                    #     print('hii')
                    
                    license_plate_images.append(license_plate_image)
        
        cv2.imshow('License Plate Detection', frame)
        writer.write(frame)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
    video_capture.release()
    writer.release()
    cv2.destroyAllWindows()
    
    return license_plate_images,extracted_text


In [30]:
license_plate_images ,extracted_text = detect_license_plates(Video_path)

In [14]:
import pandas  as pd
dicts = {}
dicts['Predicted texts From Video For Number Plates'] = extracted_text
data = pd.DataFrame(dicts)


In [ ]:
extracted_text = []
def extract_texts(image):
    for i in image:    
        if i is None:
             break
        image_for_inference = cv2.cvtColor(i, cv2.COLOR_BGR2GRAY)
        reader = easyocr.Reader(['en'],gpu=True)
        
        results = reader.readtext(image_for_inference)
        for detection in results:
                box, text, confidence = detection
                extracted_text.append(text)
        return

for i in license_plate_images:
    try:
        image_for_inference = cv2.cvtColor(i, cv2.COLOR_BGR2GRAY)
        reader = easyocr.Reader(['en'],gpu=True)
        
        results = reader.readtext(image_for_inference)
        for detection in results:
                box, text, confidence = detection
                extracted_text.append(text)
    except:
        break

In [ ]:
print(set(extracted_text))